# Reproject and Coordinate Transformation

Because of a change introduced in GDAL version >3.0, a CRS created from EPSG:4326 now expects coordinates in Y, X order instead of X, Y. The reason for this change is explained at [CRS and axis order
](https://gdal.org/tutorials/osr_api_tut.html#crs-and-axis-order).

The following code worked fine before, but now it fails with an error. In some cases, you will get results, but it will compute the re-projected coordinates with swapped X and Y coordinates

In [2]:
from osgeo import osr,ogr


wkt = 'POINT (121.787 -26.012)'

source = osr.SpatialReference()
source.ImportFromEPSG(4326)

target = osr.SpatialReference()
target.ImportFromEPSG(32750)

transform = osr.CoordinateTransformation(source, target)
point = ogr.CreateGeometryFromWkt(wkt)
point.Transform(transform)

print(point)

POINT (inf inf)


ERROR 1: PROJ: utm: Invalid latitude
ERROR 1: Invalid coordinate


To fix this, You need to explicitly set `SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)` on CRS created using EPSG:4326

In [9]:
from osgeo import osr,ogr

wkt = 'POINT (121.787 -26.012)'

source = osr.SpatialReference()
source.ImportFromEPSG(4326)
source.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)

target = osr.SpatialReference()
target.ImportFromEPSG(32750)

transform = osr.CoordinateTransformation(source, target)
point = ogr.CreateGeometryFromWkt(wkt)
point.Transform(transform)

print(point)

POINT (979368.581534574 7114191.38128267)


Higher level libraries like GeoPandas handle this under-the-hood and users do not have to worry about it.

In [7]:
import geopandas
from shapely import wkt
point = wkt.loads('POINT (121.787 -26.012)')
data = {'id': [1], 'geometry': [point]}
gdf = geopandas.GeoDataFrame(data, crs='EPSG:4326')
gdf

,id,geometry
0,1,POINT (121.78700 -26.01200)


In [8]:
gdf.to_crs(32750)

,id,geometry
0,1,POINT (979368.582 7114191.381)
